In [22]:
import pandas as pd
import json
df = pd.read_csv('../Data/twitter_raw.csv', on_bad_lines='skip')

df = df.dropna()
df.rename(columns = {df.keys()[0]:'author_id', df.keys()[1]:'cleaned_text', df.keys()[2]:'sentiment',
                    df.keys()[3]:'gcc', df.keys()[4]:'state', df.keys()[6]:'left', df.keys()[7]:'bottom',
                    df.keys()[8]:'right',df.keys()[9]:'top', df.keys()[12]:'Language'}, inplace = True)
# bbox = left,bottom,right,top = min Longitude, min Latitude, max Longitude, max Latitude
df = df.drop([df.keys()[4], df.keys()[5],df.keys()[10], df.keys()[11]], axis=1)
df.head()

,author_id,cleaned_text,sentiment,gcc,left,bottom,right,top,Language
1,45472006,life|hitting|big|change|soon|Keen|make|new|Cha...,0.156250,"{\places\"":[{\""full_name\"":\""Rockhampton","\""bbox\"":[150.459709984",-23.432698,150.580567552,-23.2462475835],en
2,952342256823943168,Sure|but|how|significant|the|warming|remains|c...,0.015385,"{\places\"":[{\""full_name\"":\""Gold Coast","\""bbox\"":[153.186551712",-28.200317,153.552171232,-27.708126368],en
3,25033901,ewwww,0.000000,"{\places\"":[{\""full_name\"":\""Toowoomba","\""bbox\"":[151.86060288",-27.654121,151.993162368,-27.493059206],en
4,3103790508,sorry|you|were|subjected|this,-0.100000,"{\places\"":[{\""full_name\"":\""Perth","\""bbox\"":[115.617614368",-32.675715,116.239023008,-31.6244855145],en
5,1249497357944672257,Awesome,0.333333,"{\places\"":[{\""full_name\"":\""Sydney","\""bbox\"":[150.520928608",-34.118347,151.343020992,-33.578140996],en


In [23]:
df['cleaned_text'] = df['cleaned_text'].str.lower()
df['gcc'] = df['gcc'].str.lower()
df['gcc'] = df['gcc'].str[29:]
#df['state'] = df['state'][1:-2].str.lower()
df['left'] = df['left'].str[10:]
df['top'] = df['top'].str.extract(r'([-]?\d+\.\d+)')

to_num_cols = ['left', 'right', 'top']
df[to_num_cols] = df[to_num_cols].astype(float)
df.head()

,author_id,cleaned_text,sentiment,gcc,left,bottom,right,top,Language
1,45472006,life|hitting|big|change|soon|keen|make|new|cha...,0.156250,rockhampton,150.459710,-23.432698,150.580568,-23.246248,en
2,952342256823943168,sure|but|how|significant|the|warming|remains|c...,0.015385,gold coast,153.186552,-28.200317,153.552171,-27.708126,en
3,25033901,ewwww,0.000000,toowoomba,151.860603,-27.654121,151.993162,-27.493059,en
4,3103790508,sorry|you|were|subjected|this,-0.100000,perth,115.617614,-32.675715,116.239023,-31.624486,en
5,1249497357944672257,awesome,0.333333,sydney,150.520929,-34.118347,151.343021,-33.578141,en


In [24]:
city_dict = {}
with open('../Data/sal.json', 'r', encoding='utf-8') as sal_file:
    sal_data = json.load(sal_file)
    for location in sal_data.keys():
        #if sal_data.get(location).get('gcc')[1] != 'r'and sal_data.get(location).get('gcc')[1] != 'R': 
        city_dict[location] = sal_data.get(location).get('gcc')
#save city_dict str into DataFrame
df_sal = pd.DataFrame({'gcc': city_dict.keys(), 
                       'Greater Capital City': city_dict.values()})
# 'state, australia' cases are not recorded in sal.json
df_sal_add = pd.DataFrame({'gcc': ['new south wales','victoria',  'queensland', 'south australia', 
                                         'western australia', 'tasmania', 'northern territory', 
                                         'australia capital territory', 'other territories'],
                           'Greater Capital City': ['1gsyd', '2gmel','3gbri','4gade', '5gper','6ghob','7gdar', 
                                                    '8acte','9oter']})
frames = [df_sal, df_sal_add]
df_sal = pd.concat(frames)
df = pd.merge(df, df_sal, on='gcc').reset_index()
df = df.drop(df.keys()[0], axis=1)
df.head()

,author_id,cleaned_text,sentiment,gcc,left,bottom,right,top,Language,Greater Capital City
0,3103790508,sorry|you|were|subjected|this,-0.100000,perth,115.617614,-32.675715,116.239023,-31.624486,en,5gper
1,1139092998,naww,0.000000,perth,115.617614,-32.675715,116.239023,-31.624486,tl,5gper
2,269765480,interesting|how|much|actual|traffic|would|you|...,0.137931,perth,115.617614,-32.675715,116.239023,-31.624486,en,5gper
3,1214733954319704064,appreciate|the|love|especially|only|available|...,0.357143,perth,115.617614,-32.675715,116.239023,-31.624486,en,5gper
4,160863117,won|has|already|been|overshadowed|deaths|and|h...,-0.269231,perth,115.617614,-32.675715,116.239023,-31.624486,en,5gper


In [26]:
df.dtypes

author_id                 int64
cleaned_text             object
sentiment               float64
gcc                      object
left                    float64
bottom                  float64
right                   float64
top                     float64
Language                 object
Greater Capital City     object
dtype: object

In [27]:
df.isnull().sum()

author_id               0
cleaned_text            0
sentiment               0
gcc                     0
left                    0
bottom                  0
right                   0
top                     0
Language                0
Greater Capital City    0
dtype: int64

In [25]:
df.to_csv('../Data/twitter_clean.csv')